### This is a test project to predict short-term stock returns using streaming data from public API

Data sources:
- Alphavantage: daily/intraday data with history of 2 years. Intraday data is delayed by a few days.
- yfinance: Intraday data is near-realtime, maybe delay of 1 min. Shorter sample window, 60d for itd data.


for more vendor api options, see:
https://nordicapis.com/10-real-time-stock-data-apis/
https://algotrading101.com/learn/yahoo-finance-api-guide/
https://algotrading101.com/learn/yfinance-guide/


Another streaming data project may be to use Google Trends data, see a template on Kaggle.

In [1]:
# !pip install yfinance

import numpy as np
import pandas as pd
import os, time, warnings, random, shap, requests, optuna
import seaborn as sns
import matplotlib.pyplot as plt
import functools as ft
import yfinance as yf


from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression, Lasso, Ridge
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV, train_test_split, KFold
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, VotingClassifier
from sklearn.metrics import accuracy_score, f1_score, r2_score, mean_squared_error
from sklearn.inspection import permutation_importance
from xgboost import XGBClassifier, XGBRegressor

pd.set_option('display.max_columns', 100)
pd.set_option('mode.chained_assignment', None)
pd.set_option('display.expand_frame_repr', False)
warnings.filterwarnings('ignore') 

time0 = time.time()

os.chdir('/home/jupyter/projects_gcp_cpu/spx/src')
os.getcwd()

'/home/jupyter/projects_gcp_cpu/spx/src'

Datapull using AV APIs:

In [26]:
data = yf.download(
        tickers = "^GSPC ^IXIC ^RUT EEM",
        period = "60d",
        interval = "2m",
        ignore_tz = True,
        group_by = 'ticker',
        auto_adjust = True,
        prepost = False,
        threads = True,
        proxy = None
    )

display(data.head(2), data.tail(6))

[*********************100%***********************]  4 of 4 completed


^GSPC                                                            ^IXIC                                                              EEM                                                 ^RUT                                              
                            Open         High          Low        Close      Volume          Open          High           Low         Close      Volume       Open   High        Low      Close     Volume         Open         High          Low        Close Volume
Datetime                                                                                                                                                                                                                                                             
2022-10-14 09:30:00  3690.409912  3699.169922  3690.409912  3693.169922  12972488.0  10745.567383  10766.067383  10742.653320  10742.653320  24871423.0  34.889999  34.93  34.869999  34.869999  1404527.0  1741.319458  1744.052734  1741.294434  1743.262085    0.0
2022-10-14 09:32:00  3693.250000  3701.979980  3689.629883  3691.979980  27010086.0  10733.133789  10754.297852  10725.332031  10725.332031  39054224.0  34.869999  34.93  34.825001  34.825001   386640.0  1743.085205  1743.250366  1741.149536  1742.499023    0.0

^GSPC                                                             ^IXIC                                                              EEM                                                     ^RUT                                              
                            Open         High          Low        Close       Volume          Open          High           Low         Close      Volume       Open       High        Low      Close     Volume         Open         High          Low        Close Volume
Datetime                                                                                                                                                                                                                                                                  
2022-11-30 15:50:00  4062.300049  4070.139893  4062.300049  4065.300049   65946000.0  11415.271484  11432.882812  11406.634766  11418.770508  61165000.0  39.590000  39.610001  39.574902  39.590000  1460648.0  1874.943970  1880.490845  1874.943970  1879.891235    0.0
2022-11-30 15:52:00  4065.139893  4069.389893  4063.790039  4067.949951   43464000.0  11417.691406  11431.492188  11414.551758  11429.641602  63361000.0  39.590000  39.599998  39.580002  39.595001   938151.0  1879.879761  1881.738770  1879.879761  1881.519409    0.0
2022-11-30 15:54:00  4068.070068  4072.280029  4067.610107  4071.429932   74778000.0  11429.517578  11449.687500  11428.899414  11446.411133  79846000.0  39.590000  39.618698  39.580002  39.599998  1910677.0  1881.212036  1884.374756  1881.158325  1883.958618    0.0
2022-11-30 15:56:00  4071.800049  4072.399902  4069.500000  4071.489990   71130000.0  11445.638672  11450.170898  11441.144531  11449.077148  75948000.0  39.599998  39.605000  39.570000  39.584999  2885228.0  1883.977051  1884.069092  1882.042236  1883.541138    0.0
2022-11-30 15:58:00  4071.149902  4078.810059  4071.149902  4078.770020  135774000.0  11446.626953  11465.195312  11445.414062  11465.195312  75132000.0  39.580002  39.584999  39.500000  39.500000  6611922.0  1883.796509  1885.863770  1883.530884  1885.863770    0.0
2022-11-30 16:00:00  4080.110107  4080.110107  4080.110107  4080.110107          0.0  11467.996094  11467.996094  11467.996094  11467.996094         0.0  39.520000  39.520000  39.520000  39.520000        0.0  1886.577271  1886.577271  1886.577271  1886.577271    0.0

In [20]:
data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9233 entries, 2022-10-14 04:36:00-04:00 to 2022-11-30 18:30:00-05:00
Data columns (total 20 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   (^IXIC, Open)    7622 non-null   float64
 1   (^IXIC, High)    7622 non-null   float64
 2   (^IXIC, Low)     7622 non-null   float64
 3   (^IXIC, Close)   7622 non-null   float64
 4   (^IXIC, Volume)  7622 non-null   float64
 5   (^GSPC, Open)    6736 non-null   float64
 6   (^GSPC, High)    6736 non-null   float64
 7   (^GSPC, Low)     6736 non-null   float64
 8   (^GSPC, Close)   6736 non-null   float64
 9   (^GSPC, Volume)  6736 non-null   float64
 10  (^RUT, Open)     6450 non-null   float64
 11  (^RUT, High)     6450 non-null   float64
 12  (^RUT, Low)      6450 non-null   float64
 13  (^RUT, Close)    6450 non-null   float64
 14  (^RUT, Volume)   6450 non-null   float64
 15  (EEM, Open)      8453 non-null   float64
 16  (EEM, High) 

In [21]:
display(data.shape, data.iloc[180:183,])

(9233, 20)

^IXIC                                                              ^GSPC                                                           ^RUT                                                      EEM                                          
                                   Open          High           Low         Close      Volume         Open         High          Low        Close     Volume         Open         High          Low        Close Volume       Open       High        Low      Close   Volume
Datetime                                                                                                                                                                                                                                                                    
2022-10-14 14:10:00-04:00  10401.321289  10401.321289  10395.275391  10396.036133   6456000.0  3603.379883  3603.379883  3601.340088  3602.209961  6362000.0  1695.246338  1695.246338  1694.673096  1694.673096    0.0  34.369999  34.369999  34.360001  34.368500  55080.0
2022-10-14 14:12:00-04:00  10389.238281  10390.500977  10383.914062  10383.914062   9034000.0  3599.239990  3599.899902  3597.729980  3597.800049  7611000.0  1694.671265  1694.671265  1694.092529  1694.092529    0.0  34.365002  34.365002  34.340000  34.340000  81080.0
2022-10-14 14:14:00-04:00  10383.956055  10387.025391  10382.614258  10383.316406  10513000.0  3597.780029  3599.229980  3597.590088  3597.590088  7024000.0  1693.947876  1693.947876  1693.605591  1693.605591    0.0  34.345001  34.360001  34.345001  34.349998  53933.0

In [22]:
data.columns

MultiIndex([('^IXIC',   'Open'),
            ('^IXIC',   'High'),
            ('^IXIC',    'Low'),
            ('^IXIC',  'Close'),
            ('^IXIC', 'Volume'),
            ('^GSPC',   'Open'),
            ('^GSPC',   'High'),
            ('^GSPC',    'Low'),
            ('^GSPC',  'Close'),
            ('^GSPC', 'Volume'),
            ( '^RUT',   'Open'),
            ( '^RUT',   'High'),
            ( '^RUT',    'Low'),
            ( '^RUT',  'Close'),
            ( '^RUT', 'Volume'),
            (  'EEM',   'Open'),
            (  'EEM',   'High'),
            (  'EEM',    'Low'),
            (  'EEM',  'Close'),
            (  'EEM', 'Volume')],
           )

In [24]:
data.columns = data.columns.to_flat_index()
data.head()

,"(^IXIC, Open)","(^IXIC, High)","(^IXIC, Low)","(^IXIC, Close)","(^IXIC, Volume)","(^GSPC, Open)","(^GSPC, High)","(^GSPC, Low)","(^GSPC, Close)","(^GSPC, Volume)","(^RUT, Open)","(^RUT, High)","(^RUT, Low)","(^RUT, Close)","(^RUT, Volume)","(EEM, Open)","(EEM, High)","(EEM, Low)","(EEM, Close)","(EEM, Volume)"
Datetime,,,,,,,,,,,,,,,,,,,,
2022-10-14 04:36:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.75,34.75,34.75,34.75,0.0
2022-10-14 04:52:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.77,34.77,34.77,34.77,0.0
2022-10-14 05:08:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.69,34.69,34.69,34.69,0.0
2022-10-14 07:02:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.85,34.85,34.85,34.85,0.0
2022-10-14 07:10:00-04:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,34.94,34.94,34.94,34.94,0.0


In [27]:
tickerStrings = ['^GSPC', '^IXIC', '^RUT', 'EEM']
df_list = list()
for ticker in tickerStrings:
    data = yf.download(ticker, group_by="Ticker", period='60d', interval='2m', prepost=False, auto_adjust=True)
    data['ticker'] = ticker  
    df_list.append(data)

df = pd.concat(df_list)
df

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Volume,ticker
Datetime,,,,,,
2022-10-14 09:30:00,3690.409912,3699.169922,3690.409912,3693.169922,12972488,^GSPC
2022-10-14 09:32:00,3693.250000,3701.979980,3689.629883,3691.979980,27010086,^GSPC
2022-10-14 09:34:00,3691.489990,3694.030029,3686.590088,3689.870117,22377176,^GSPC
2022-10-14 09:36:00,3689.520020,3699.040039,3689.520020,3698.219971,18554301,^GSPC
2022-10-14 09:38:00,3698.409912,3712.000000,3696.879883,3711.330078,20848550,^GSPC
...,...,...,...,...,...,...
2022-11-30 15:52:00,39.590000,39.599998,39.580002,39.595001,938151,EEM
2022-11-30 15:54:00,39.590000,39.618698,39.580002,39.599998,1910677,EEM
2022-11-30 15:56:00,39.599998,39.605000,39.570000,39.584999,2885228,EEM


In [29]:
df = df[['Close', 'ticker']]
df

,Close,ticker
Datetime,,
2022-10-14 09:30:00,3693.169922,^GSPC
2022-10-14 09:32:00,3691.979980,^GSPC
2022-10-14 09:34:00,3689.870117,^GSPC
2022-10-14 09:36:00,3698.219971,^GSPC
2022-10-14 09:38:00,3711.330078,^GSPC
...,...,...
2022-11-30 15:52:00,39.595001,EEM
2022-11-30 15:54:00,39.599998,EEM
2022-11-30 15:56:00,39.584999,EEM


In [30]:
df.ticker.value_counts()

^GSPC    6342
^IXIC    6341
^RUT     6341
EEM      6335
Name: ticker, dtype: int64

In [36]:
df.replace({'^GSPC':'Spx', '^IXIC':'Nasdaq', '^RUT':'Russel'}, inplace=True)
df

,Close,ticker
Datetime,,
2022-10-14 09:30:00,3693.169922,Spx
2022-10-14 09:32:00,3691.979980,Spx
2022-10-14 09:34:00,3689.870117,Spx
2022-10-14 09:36:00,3698.219971,Spx
2022-10-14 09:38:00,3711.330078,Spx
...,...,...
2022-11-30 15:52:00,39.595001,EEM
2022-11-30 15:54:00,39.599998,EEM
2022-11-30 15:56:00,39.584999,EEM


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 25359 entries, 2022-10-14 09:30:00 to 2022-11-30 16:00:00
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Close   25359 non-null  float64
 1   ticker  25359 non-null  object 
dtypes: float64(1), object(1)
memory usage: 594.4+ KB


In [38]:
df.describe()

,Close
count,25359.000000
mean,4155.864696
std,4132.246578
min,33.540001
25%,1685.485291
50%,3585.729980
75%,10270.178223
max,11488.499023


In [ ]:
data = yf.download(  # or pdr.get_data_yahoo(...
        # tickers list or string as well
        tickers = "^GSPC ^IXIC ^RUT EEM",

        # use "period" instead of start/end
        # valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max
        # (optional, default is '1mo')
        period = "60d",

        # fetch data by interval (including intraday if period < 60 days)
        # valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo
        # (optional, default is '1d')
        interval = "2m",

        # Whether to ignore timezone when aligning ticker data from 
        # different timezones. Default is True. False may be useful for 
        # minute/hourly data.
        ignore_tz = True,

        # group by ticker (to access via data['SPY'])
        # (optional, default is 'column')
        group_by = 'ticker',

        # adjust all OHLC automatically
        # (optional, default is False)
        auto_adjust = True,

        # download pre/post regular market hours data
        # (optional, default is False)
        prepost = False,

        # use threads for mass downloading? (True/False/Integer)
        # (optional, default is True)
        threads = True,

        # proxy URL scheme use use when downloading?
        # (optional, default is None)
        proxy = None
    )

display(data.head(2), data.tail(6))

In [4]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=SPY&interval=5min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
#adjusted=true&
spy = pd.read_csv(url)
spy

# at 9:50am monday there is still no data after friday close.
# apparently, there is one calendar day of delay.

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEM&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eem = pd.read_csv(url)
print(eem.shape)

In [16]:
url = 'https://www.alphavantage.co/query?function=CRYPTO_INTRADAY&symbol=ETH&market=USD&interval=5min&apikey=KBYUCPQEHAG67WNC'
eth = pd.read_csv(url)
eth

,{
0,"""Information"": ""Thank you for using Alpha ..."
1,}


In [12]:
eth[0:30]

,{
0,"""Information"": ""Thank you for using Alpha ..."
1,}


In [15]:
eth.iloc[0,0]

'    "Information": "Thank you for using Alpha Vantage! This is a premium endpoint. You may subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly unlock all premium endpoints"'

In [ ]:
# can try predicting 10 yield, try smth like xgb 200, 4, 0.04

In [67]:
# it is hard to get any positive results at all using daily freq major assets
# can try to exploit intraday lead-lag effects
# i.e., pick up intraday major assets (3 indices) and try to predict less liquid assets.
# can try btc or etfs of small stocks. e.g., eem or eems etfs. 

In [88]:
url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEM&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eem = pd.read_csv(url)
print(eem.shape)

# EEMS is even less liquid, try eema emxc

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEMS&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eems = pd.read_csv(url)
print(eems.shape)

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EWX&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
ewx = pd.read_csv(url)
print(ewx.shape)

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EEMA&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
eema = pd.read_csv(url)
print(eema.shape)

url = 'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY_EXTENDED&symbol=EMXC&interval=1min&slice=year1month1&apikey=KBYUCPQEHAG67WNC&datatype=csv'
emxc = pd.read_csv(url)
print(emxc.shape)

(9752, 6)
(1210, 6)
(1230, 6)
(1384, 6)
(6033, 6)


In [87]:
display(spy[192:250], eem[:50], emxc[:50])

,time,open,high,low,close,volume
192,2022-11-21 16:01:00,394.5800,394.6900,394.5300,394.6200,195089
193,2022-11-21 16:00:00,394.4900,394.7900,394.4800,394.5800,1210352
194,2022-11-21 15:59:00,394.4250,394.5000,394.3800,394.4900,686851
195,2022-11-21 15:58:00,394.4400,394.5000,394.3900,394.4250,255209
196,2022-11-21 15:57:00,394.7200,394.7200,394.4200,394.4345,284264
197,2022-11-21 15:56:00,394.8000,394.8000,394.6300,394.7000,278622
198,2022-11-21 15:55:00,395.0600,395.1200,394.7700,394.8000,354828
199,2022-11-21 15:54:00,394.9000,395.0800,394.8300,395.0600,283090
200,2022-11-21 15:53:00,394.7500,394.9200,394.7400,394.9000,147763
201,2022-11-21 15:52:00,394.6300,394.9200,394.5900,394.7500,188787


,time,open,high,low,close,volume
0,2022-11-21 19:18:00,37.550,37.5500,37.5500,37.5500,206
1,2022-11-21 19:06:00,37.550,37.5500,37.5500,37.5500,1000
2,2022-11-21 18:46:00,37.550,37.5500,37.5500,37.5500,145
3,2022-11-21 18:43:00,37.550,37.5500,37.5500,37.5500,270
4,2022-11-21 18:01:00,37.550,37.5500,37.5500,37.5500,461
5,2022-11-21 17:51:00,37.550,37.5500,37.5500,37.5500,100
6,2022-11-21 17:00:00,37.550,37.5500,37.5500,37.5500,2000
7,2022-11-21 16:34:00,37.530,37.5300,37.5200,37.5200,4600
8,2022-11-21 16:30:00,37.550,37.5500,37.5400,37.5400,9599
9,2022-11-21 16:26:00,37.550,37.5500,37.5500,37.5500,7501


,time,open,high,low,close,volume
0,2022-11-21 16:00:00,48.7810,48.8000,48.7600,48.7600,5387
1,2022-11-21 15:59:00,48.7800,48.7800,48.7745,48.7745,874
2,2022-11-21 15:58:00,48.7700,48.7725,48.7700,48.7725,993
3,2022-11-21 15:56:00,48.7892,48.7892,48.7700,48.7700,559
4,2022-11-21 15:55:00,48.7601,48.7601,48.7601,48.7601,422
5,2022-11-21 15:54:00,48.7601,48.7900,48.7601,48.7900,2033
6,2022-11-21 15:52:00,48.7850,48.7850,48.7850,48.7850,212
7,2022-11-21 15:51:00,48.7800,48.7800,48.7700,48.7700,334
8,2022-11-21 15:50:00,48.7800,48.7850,48.7800,48.7850,8183
9,2022-11-21 15:46:00,48.7500,48.7700,48.7500,48.7700,597


In [17]:
# fix date, create returns and rate differences over 1d, 5d, 21d, 63d, 121d, 252d, 504d for all 4 variables
# then create signals by lagging evth by 1d
# then clean evth and fir xgb (around 30 features)

#### step 1:
build simple XGB model
#### step 2:
deploy this model via Cloud Run and static web app, fixed model
#### step 3
deploy model via Cloud Run and Flask with dynamic model, retrained daily